# Required Packages

In [1]:
import os
import re

# Loading DataSet

In [8]:
def loadDataset(dir):
    data = []
    for file in os.listdir( dir ):
        try:
            with open( os.path.join( dir, file ) ,"r") as f:
                data.append( f.read() )
        except :
            print("could not read file")
    return data

Loading Train

In [9]:
reviews_train = []
train_dir = "../DataSet/aclImdb/train"
reviews_train  = loadDataset(os.path.join( train_dir, "pos" ))
reviews_train += loadDataset(os.path.join( train_dir, "neg" ))

Loading Test

In [15]:
reviews_test = []
test_dir = "../DataSet/aclImdb/test"
reviews_test  = loadDataset(os.path.join( test_dir, "pos" ))
reviews_test += loadDataset(os.path.join( test_dir, "neg" ))

# Cleaning Dataset
Removing HTML tags.